In [1]:
from __future__ import print_function

import os
import csv
import random
import time
from datetime import datetime

In [4]:
# toplogy_t_solution_mlu_result = 'toplogy_t_solution_mlu_result.csv'

# # topology_name = 'abilene'

# # ['topology_name','time/trafic_matrix_id','flow_id','from_node','to_node','ratio','optimal_mlu']
# counter = 0
# tm = set([])
# each_topology_each_tm_paths = {}
# all_topologies = set([])
# with open(toplogy_t_solution_mlu_result, "r") as f:
#     reader = csv.reader(f, delimiter=",")
#     for line in (reader):
#         if line[0] !='topology_name':
#             if 'ATT' in line[0]:
#                 counter +=1
#                 tm.add(line[1])
# print('att last line number %s and has %s tm '%(counter,len(list(tm))))

att last line number 13440000 and has 200 tm 


In [4]:
def remove_zero_ratio_lines():
    toplogy_t_solution_mlu_result = 'toplogy_t_solution_mlu_result.csv'
    toplogy_t_solution_mlu_purified_result = 'toplogy_t_solution_mlu_purified_result.csv'
    all_lines = []
    line_counter = 0
    error = 0
    with open(toplogy_t_solution_mlu_result, "r") as f:
        reader = csv.reader(f, delimiter=",")
        for line in (reader):
            line_counter+=1
            try:
                if float(line[5]) ==0.0:
                    all_lines.append(line)
            except:
                error+=1
                pass
    with open(toplogy_t_solution_mlu_purified_result, 'a') as newFile:                                
        newFileWriter = csv.writer(newFile)
        for l  in all_lines:
            #print('flow index %s from node %s to the next node %s ratio %s'%(item[0],item[1],item[2],v))
            newFileWriter.writerow([i for i in l]) #50_04:00:05.606476,0,1000,"3320,6724",link_down_reverse
    print('we reduced %s to %s error %s'%(line_counter,len(all_lines),error))
    
#remove_zero_ratio_lines()
print('done!')

we reduced 21423362 to 20070272 error 2
done!


In [5]:


def check_path_already_added(a_new_path,set_of_paths):
    
    for path in set_of_paths:
        if path == a_new_path:
            return True
        
    return False

def check_if_same_path(tm,flow,each_tm_each_flow_all_edges):
    all_current_tm_edges = []
    for l in each_tm_each_flow_all_edges[tm][flow]:
        all_current_tm_edges.append(l)
    all_current_tm_edges.sort()
    for p_t in range(0,int(tm)-1):
        #print('for history ',p_t)
#         print('all_edges_for current t',all_current_tm_edges)
#         print('previous t this flow edges',each_tm_each_flow_all_edges[str(p_t)][flow],type(each_tm_each_flow_all_edges[str(p_t)][flow]))
#         print('done')
        previous_t_edges = each_tm_each_flow_all_edges[str(p_t)][flow]
        previous_t_edges.sort()
        if all_current_tm_edges == previous_t_edges:
            #print('we are returning True')
            return True, str(p_t)
#         else:
#             print('current',all_current_tm_edges)
#             print('previous',previous_t_edges)
#     print('we are returning False')
    return False,None
        


toplogy_t_solution_mlu_result = 'test.csv'
toplogy_t_solution_mlu_result = 'new_toplogy_t_solution_mlu_result.csv'

# topology_name = 'abilene'

# ['topology_name','time/trafic_matrix_id','flow_id','from_node','to_node','ratio','optimal_mlu']
each_topology_each_tm_paths = {}
all_topologies = set([])
with open(toplogy_t_solution_mlu_result, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in (reader):
        if line[0] !='topology_name':
            all_topologies.add(line[0])
        
print(' we got all topologies',all_topologies)
each_topology_flow_ID_file = {"B4":"data/B4_shortest_paths",'Abilene':'abilene_flow.txt','ATT_topology_file_modified':'ATT_topology_file_modified_shortest_paths.txt'}

#for topology in ['Abilene','ATT_topology_file_modified']:
for topology in ['B4']:
    
    
    
    each_flow_source_dst ={}
    """reading source destination of each flow from topology file"""
    flow_index = 0
    print('reading flow ids')
    #with open('test.txt') as f:
    #with open('att_flows.txt') as f:
    with open(each_topology_flow_ID_file[topology]) as f:
        lines = f.readlines()
        for line in lines:
            #print(line)
            src = line.split(":")[0].split("->")[0]
            dst = line.split(":")[0].split("->")[1]
            each_flow_source_dst[str(flow_index)] = {}
            each_flow_source_dst[str(flow_index)]['src'] = src
            each_flow_source_dst[str(flow_index)]['dst'] = dst
            flow_index+=1

    all_tms = set([])
    each_topology_each_tm_paths[topology] = {}
    with open(toplogy_t_solution_mlu_result, "r") as f:
        reader = csv.reader(f, delimiter=",")
        for line in (reader):
            if line[0] == topology:
                all_tms.add(line[1])
    print('we got all tms',len(list(all_tms)))
    all_tms = list(all_tms)
    all_tms.sort()
    integers_all_tms = []
    for t in all_tms:
        integers_all_tms.append(int(t))
    integers_all_tms.sort()
    each_tm_each_flow_all_edges = {}
    each_tm_each_flow_paths = {}
    for tm in integers_all_tms:
        if tm <=1 or 1==1:
            tm = str(tm)
            each_tm_each_flow_all_edges[tm] = {}
            each_tm_each_flow_paths[tm] = {}
            now = datetime.now()

            current_time = now.strftime("%H:%M:%S")

            print('for tm %s from %s Current Time = %s'%(tm,len(list(all_tms)),current_time))
            flows = set([])

            each_topology_each_tm_paths[topology][tm] = {}
            each_flow_each_node_out_neighbors = {}
            with open(toplogy_t_solution_mlu_result, "r") as f:
                reader = csv.reader(f, delimiter=",")
                for line in (reader):
                    if line[0]==topology and line[1] ==tm:
                        if line[5]!='flow_id':
                            flows.add(line[2])
                            if line[2] not in each_flow_each_node_out_neighbors:
                                each_flow_each_node_out_neighbors[line[2]] = {}

                        try:
#                             if line[2]=='51':
#                                 print('this is the line for flow 100',line)
                            
                            if float(line[5])!=0.0:
                                
                                try:
                                    each_flow_each_node_out_neighbors[line[2]][line[3]].append(line[4])
                                    #print(line[4],line[3],'was added')
                                except:
                                    each_flow_each_node_out_neighbors[line[2]][line[3]]=[line[4]]   
                                try:
                                    each_tm_each_flow_all_edges[tm][line[2]].append([line[3],line[4]])
                                except:
                                    each_tm_each_flow_all_edges[tm][line[2]] = []
                                    each_tm_each_flow_all_edges[tm][line[2]].append([line[3],line[4]])
                        except ValueError:
                            print('ValueError',ValueError)
                            pass
            all_flows_all_paths = {}

            #print('read all flows',each_flow_source_dst)
            flows = list(flows)
            flows.sort()
            #print('flows',flows)
            #print('each_flow_source_dst',each_flow_source_dst)
            flows = list(flows)
            flows.sort()
            integer_flows = []
            for f in flows:
                integer_flows.append(int(f))
            integer_flows.sort()

            for each_flow in integer_flows:
                current_time = time.time()
                each_flow = str(each_flow)

                #if each_flow =='79':
                    #print(each_flow_source_dst[each_flow])
                    #print(each_flow_each_node_out_neighbors[each_flow])
                #print("******* for flow %s from all %s for tm %s****"%(each_flow,len(flows),tm))
                set_of_paths = []
                checked_edges = []
                got_it = False
                if int(tm) >0:
                    got_it,p_t =  check_if_same_path(tm,each_flow,each_tm_each_flow_all_edges)
                    if got_it:
                        #print('got from history!!!')
                        set_of_paths = each_tm_each_flow_paths[p_t][each_flow]


                while(not got_it and len(list(checked_edges))<len(each_tm_each_flow_all_edges[tm][each_flow])):
                    #print('building new path')
                    if time.time()- current_time>4:
                        break
                    a_new_path = []
                    flaw_in_solution = False
                    #print(each_flow_source_dst)
                    src = each_flow_source_dst[each_flow]['src'] 
                    dst = each_flow_source_dst[each_flow]['src']
                    a_new_path.append(src)
#                     if each_flow_source_dst[each_flow]['src'] =='4' and each_flow_source_dst[each_flow]['dst']=='8':
#                         print(each_flow_source_dst[each_flow]['src'],each_flow_source_dst[each_flow]['dst'],each_tm_each_flow_all_edges[tm][each_flow])
#                         import pdb
#                         #pdb.set_trace()
#                     elif each_flow_source_dst[each_flow]['src'] =='11' and each_flow_source_dst[each_flow]['dst']=='7':
#                         print(each_flow_source_dst[each_flow]['src'],each_flow_source_dst[each_flow]['dst'],each_tm_each_flow_all_edges[tm][each_flow])
#                         import pdb
                        #pdb.set_trace()
                    while(dst!=each_flow_source_dst[each_flow]['dst'] and not flaw_in_solution):
                        try:
                            next_node = each_flow_each_node_out_neighbors[each_flow][dst][random.randint(0,len(each_flow_each_node_out_neighbors[each_flow][dst])-1)]
                            a_new_path.append(next_node)
                            dst = next_node
                        except:
                            flaw_in_solution = True
                            #pdb.set_trace()


                    if not flaw_in_solution and not check_path_already_added(a_new_path,set_of_paths):
                        #print('new path added')
                        if (len(set(a_new_path))==len(a_new_path)):
                            set_of_paths.append(a_new_path)
                            for i in range(len(a_new_path)-1):
                                if [a_new_path[i],a_new_path[i+1]] not in checked_edges:
                                    checked_edges.append([a_new_path[i],a_new_path[i+1]])
    #                 else:
    #                     if  flaw_in_solution:
    #                         print('flaw in solution')
                            #print('each_flow_each_node_out_neighbors[each_flow]',each_flow_each_node_out_neighbors[each_flow])
    #                     else:
    #                         print('already added path!')
                all_flows_all_paths[each_flow] = set_of_paths

                each_tm_each_flow_paths[tm][each_flow] = set_of_paths
                
                paths_string = topology+':'+str(tm)+':'+each_flow_source_dst[each_flow]['src']+'->'+each_flow_source_dst[each_flow]['dst']+':'
                flag = True
#                 if each_flow_source_dst[each_flow]['src']=='4' and each_flow_source_dst[each_flow]['dst'] =='8':
#                     print('for ',set_of_paths)
#                     import pdb
                    #pdb.set_trace()
                for p in set_of_paths:
                    if flag:
                        paths_string = paths_string +' '+str(p)
                        flag = False
                    else:
                        paths_string = paths_string+','+str(p)
                file_object = open('new_each_topology_each_t_each_f_paths.txt', 'a')
                # Append 'hello' at the end of file
                file_object.write(paths_string+"\n")
                # Close the file
                file_object.close()
#                 if each_flow_source_dst[each_flow]['src']=='4' and each_flow_source_dst[each_flow]['dst'] =='8':
#                     print('for ',set_of_paths)
#                     print('these are the edges',each_tm_each_flow_all_edges[tm][each_flow])
#                     print('this is the flow number ',each_flow)
#                     import pdb
#                     pdb.set_trace()
            
#                  7->19: [[7, 4, 12, 11, 19], [7, 4, 6, 1, 11, 19]]
        each_topology_each_tm_paths[topology][tm] = all_flows_all_paths
        
        
#print('each_topology_each_tm_paths',each_topology_each_tm_paths)
        

 we got all topologies {'B4'}
reading flow ids
we got all tms 199
for tm 0 from 199 Current Time = 21:56:38
for tm 1 from 199 Current Time = 21:57:07
for tm 2 from 199 Current Time = 21:57:09
for tm 3 from 199 Current Time = 21:57:10
for tm 4 from 199 Current Time = 21:57:11
for tm 5 from 199 Current Time = 21:57:13
for tm 6 from 199 Current Time = 21:57:14
for tm 7 from 199 Current Time = 21:57:15
for tm 8 from 199 Current Time = 21:57:56
for tm 9 from 199 Current Time = 21:57:58
for tm 10 from 199 Current Time = 21:58:00
for tm 11 from 199 Current Time = 21:58:10
for tm 12 from 199 Current Time = 21:58:20
for tm 13 from 199 Current Time = 21:58:21
for tm 14 from 199 Current Time = 21:58:38
for tm 15 from 199 Current Time = 21:58:55
for tm 16 from 199 Current Time = 21:58:57
for tm 17 from 199 Current Time = 21:59:14
for tm 18 from 199 Current Time = 21:59:19
for tm 19 from 199 Current Time = 21:59:33
for tm 20 from 199 Current Time = 21:59:50
for tm 21 from 199 Current Time = 21:59:5

for tm 188 from 199 Current Time = 22:07:06
for tm 189 from 199 Current Time = 22:07:08
for tm 190 from 199 Current Time = 22:07:10
for tm 191 from 199 Current Time = 22:07:12
for tm 192 from 199 Current Time = 22:07:13
for tm 193 from 199 Current Time = 22:07:15
for tm 194 from 199 Current Time = 22:07:16
for tm 195 from 199 Current Time = 22:07:22
for tm 196 from 199 Current Time = 22:07:24
for tm 197 from 199 Current Time = 22:07:26
for tm 198 from 199 Current Time = 22:07:28


In [ ]:
for j in range(2,3):
    print(j)

In [12]:
import time
current_time = time.time()


In [26]:
print(current_time)
print(time.time())
if time.time()- current_time>30:
    print(time.time()- current_time)
else:
    print( '2',time.time()- current_time)

1647110276.740213
1647110307.836093
31.096117973327637


In [30]:
if ['3','2','5'] in ['5','2','3']:
    print('True')
else:
    print('False')

False


In [29]:
120*3/60

6.0

In [33]:
my_list = [['0', '1']]


TypeError: argument of type 'NoneType' is not iterable

ModuleNotFoundError: No module named 'pyscipopt'

In [ ]:
# conda create --name myenv
# conda activate myenv
# conda install --channel conda-forge pyscipopt
# python -m pytest